# GLGENN on Colored Fashion-MNIST

In [7]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.12/dist-packages/google/colab/drive.py", line 97, in mount
    return _mount(
           ^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/colab/drive.py", line 134, in _mount
    _message.blocking_request(
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_message.py", line 173, in blocking_request
    request_id = send_request(
                 ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_message.py", line 117, in send_request
    instance = ipython.get_kernelapp()
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_ipython.py", line 28, in get_kernelapp
    return get_ipython().kernel.parent
           ^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'kernel'


In [ ]:
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets
import torchvision.transforms.v2 as v2

from glgenn.algebra.cliffordalgebraex import CliffordAlgebraQT
from glgenn.algebra.cliffordalgebra import CliffordAlgebra
from glgenn.layers.qtgp import QTGeometricProduct
from glgenn.layers.qtlinear import QTLinear
from glgenn.layers.qtnorm import QTNormalization

import matplotlib.pyplot as plt


In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

## Dataloader (Colored Fashion MNIST + GA embedding)


In [5]:
class CliffordFashionedMnist(Dataset):
    def __init__(self, root, train=True, download=True, d=5):
        self.metric = [1] * d
        self.ca = CliffordAlgebraQT(self.metric)
        self.d = d
        self.h, self.w = 28, 28
        self.post_transforms = v2.Compose([
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.data = datasets.FashionMNIST(root, train=train, download=download)

        y_coords, x_coords = torch.meshgrid(
            torch.linspace(-1, 1, self.h),
            torch.linspace(-1, 1, self.w),
            indexing="ij"
        )
        self.grid = torch.stack([x_coords, y_coords], dim=0) # [2, 28, 28]

    def _colorize_random(self, img):
        img_np = np.array(img) 
        factors = np.random.uniform(0.2, 1.0, 3)
        color_img = np.stack([img_np * f for f in factors], axis=-1).astype(np.uint8) # [28, 28, 3]
        return color_img

    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        img_raw, label = self.data[idx]
        img_colored = self._colorize_random(img_raw) # [3, 28, 28]
        img_tensor = self.post_transforms(img_colored) 
        v_full = torch.cat([self.grid, img_tensor], dim=0) # [5, 28, 28]
        v_full = v_full.permute(1, 2, 0) # [28, 28, 5]
        x_mv = self.ca.embed_grade(v_full, 1)  # [28, 28, 2 ** 5]
        return x_mv, label

### Prepare: MVSiLU from CGENN 

In [6]:
def unsqueeze_like(tensor: torch.Tensor, like: torch.Tensor, dim=0):
    """
    Unsqueeze last dimensions of tensor to match another tensor's number of dimensions.

    Args:
        tensor (torch.Tensor): tensor to unsqueeze
        like (torch.Tensor): tensor whose dimensions to match
        dim: int: starting dim, default: 0.
    """
    n_unsqueezes = like.ndim - tensor.ndim
    if n_unsqueezes < 0:
        raise ValueError(f"tensor.ndim={tensor.ndim} > like.ndim={like.ndim}")
    elif n_unsqueezes == 0:
        return tensor
    else:
        return tensor[dim * (slice(None),) + (None,) * n_unsqueezes]

class MVSiLU(nn.Module):
    def __init__(self, algebra, channels, invariant="mag2", exclude_dual=False):
        super().__init__()
        self.algebra = algebra
        self.channels = channels
        self.exclude_dual = exclude_dual
        self.invariant = invariant
        self.a = nn.Parameter(torch.ones(1, channels, algebra.dim + 1))
        self.b = nn.Parameter(torch.zeros(1, channels, algebra.dim + 1))

        if invariant == "norm":
            self._get_invariants = self._norms_except_scalar
        elif invariant == "mag2":
            self._get_invariants = self._mag2s_except_scalar
        else:
            raise ValueError(f"Invariant {invariant} not recognized.")

    def _norms_except_scalar(self, input):
        return self.algebra.norms(input, grades=self.algebra.grades[1:])

    def _mag2s_except_scalar(self, input):
        return self.algebra.qs(input, grades=self.algebra.grades[1:])

    def forward(self, input):
        norms = self._get_invariants(input)
        norms = torch.cat([input[..., :1], *norms], dim=-1)
        a = unsqueeze_like(self.a, norms, dim=2)
        b = unsqueeze_like(self.b, norms, dim=2)
        norms = a * norms + b
        norms = norms.repeat_interleave(self.algebra.subspaces, dim=-1)
        return torch.sigmoid(norms) * input

## Model 


In [7]:
class CGEBlock(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()

        self.layers = nn.Sequential(
            QTLinear(algebra, in_features, out_features),
            MVSiLU(algebra, out_features),
            QTGeometricProduct(algebra, out_features),
            QTNormalization(algebra, out_features)
        )

    def forward(self, input):
        # [batch_size, in_features, 2**d] -> [batch_size, out_features, 2**d]
        return self.layers(input)

In [8]:
class CGEMLP(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_layers=2):
        super().__init__()

        layers = []
        for i in range(n_layers - 1):
            layers.append(
                CGEBlock(algebra, in_features, hidden_features)
            )
            in_features = hidden_features

        layers.append(
            CGEBlock(algebra, hidden_features, out_features)
        )
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)

In [9]:
class CliffordFashionModel(nn.Module):
    def __init__(self, ca, in_channels=1, hidden_channels=16, out_classes=10):
        super().__init__()
        self.ca = ca
        self.cge_part = CGEMLP(ca, in_channels, hidden_channels, hidden_channels)
        self.activation = MVSiLU(ca, hidden_channels)
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, out_classes)
        )

    def forward(self, x):
        batch_size, h, w, mv_dim = x.shape
        print(f"Debug: Input shape {x.shape}") # Проверка входящего размера
        
        x = x.view(batch_size * h * w, 1, mv_dim)
        print(f"Debug: After view {x.shape}")
        
        # Проверяем проход через основной блок
        print("Debug: Entering CGEMLP...")
        h_out = self.cge_part(x) 
        print("Debug: CGEMLP finished.")
        
        h_out = self.activation(h_out)
        invariants = h_out[..., 0]
        invariants = invariants.view(batch_size, h * w, -1)
        
        print("Debug: Global average pooling...")
        pooled = invariants.mean(dim=1)
        
        return self.classifier(pooled)

## Training & Evaluating 

In [ ]:
import psutil

def print_memory_stats(step_name):
    process = psutil.Process()
    ram_usage = process.memory_info().rss / 1024 / 1024  # в МБ
    vram_stats = ""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024 / 1024
        reserved = torch.cuda.memory_reserved() / 1024 / 1024
        vram_stats = f" | VRAM Allocated: {allocated:.1f}MB, Reserved: {reserved:.1f}MB"
    print(f"[{step_name}] RAM: {ram_usage:.1f}MB{vram_stats}")

In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.set_default_device('cuda')

print(f"Device: {device}")

d = 5
lr = 0.001
epochs = 5
batch_size = 64

ca = CliffordAlgebraQT([1] * d)

if device.type == 'cuda':
    if hasattr(ca, 'qt_geometric_product_paths'):
        ca.qt_geometric_product_paths = ca.qt_geometric_product_paths.to(device)

full_dataset = CliffordFashionedMnist(root='./data', train=True, download=True)
train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True, generator=torch.Generator(device='cpu'))

print("Debug: Testing DataLoader...")
test_batch, test_labels = next(iter(train_loader))
print(f"Debug: Batch shape: {test_batch.shape}")

model = CliffordFashionModel(ca, in_channels=1, hidden_channels=16).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
    
    for batch_idx, (images, labels) in enumerate(pbar):
        print(f"\n--- Batch {batch_idx} Start ---")
        print_memory_stats("Before Loading to Device")
        
        images, labels = images.to(device), labels.to(device)
        print_memory_stats("After Loading to Device")
        
        optimizer.zero_grad()
        
        # Замеряем Forward pass по частям
        print("Starting Forward pass...")
        outputs = model(images)
        print_memory_stats("After Forward pass")
        
        loss = criterion(outputs, labels)
        
        print("Starting Backward pass...")
        loss.backward()
        print_memory_stats("After Backward pass")
        
        optimizer.step()
        print("--- Batch End ---\n")

Device: cpu
Debug: Testing DataLoader...
Debug: Batch shape: torch.Size([64, 28, 28, 32])


Epoch 1/5:   0%|          | 0/938 [00:00<?, ?it/s]


--- Batch 0 Start ---
[Before Loading to Device] RAM: 258.5MB
[After Loading to Device] RAM: 258.7MB
Starting Forward pass...
Debug: Input shape torch.Size([64, 28, 28, 32])
Debug: After view torch.Size([50176, 1, 32])
Debug: Entering CGEMLP...
Debug: CGEMLP finished.
Debug: Global average pooling...
[After Forward pass] RAM: 734.4MB
Starting Backward pass...


Epoch 1/5:   0%|          | 1/938 [00:35<9:13:32, 35.45s/it]

[After Backward pass] RAM: 1329.8MB
--- Batch End ---


--- Batch 1 Start ---
[Before Loading to Device] RAM: 1259.5MB
[After Loading to Device] RAM: 1259.5MB
Starting Forward pass...
Debug: Input shape torch.Size([64, 28, 28, 32])
Debug: After view torch.Size([50176, 1, 32])
Debug: Entering CGEMLP...
Debug: CGEMLP finished.
Debug: Global average pooling...
[After Forward pass] RAM: 1013.8MB
Starting Backward pass...


...

[After Backward pass] RAM: 1303.4MB
--- Batch End ---


--- Batch 7 Start ---
[Before Loading to Device] RAM: 1222.7MB
[After Loading to Device] RAM: 1222.7MB
Starting Forward pass...
Debug: Input shape torch.Size([64, 28, 28, 32])
Debug: After view torch.Size([50176, 1, 32])
Debug: Entering CGEMLP...
Debug: CGEMLP finished.
Debug: Global average pooling...
[After Forward pass] RAM: 695.1MB
Starting Backward pass...


Epoch 1/5:   1%|          | 8/938 [05:04<9:20:33, 36.16s/it]

[After Backward pass] RAM: 1193.1MB
--- Batch End ---


--- Batch 8 Start ---
[Before Loading to Device] RAM: 937.7MB
[After Loading to Device] RAM: 937.3MB
Starting Forward pass...
Debug: Input shape torch.Size([64, 28, 28, 32])
Debug: After view torch.Size([50176, 1, 32])
Debug: Entering CGEMLP...


Epoch 1/5:   1%|          | 8/938 [05:11<10:02:55, 38.90s/it]


KeyboardInterrupt: 

## Building standard models

## Stress tests

## Results